In [3]:
import csv
import pandas as pd
from dateutil.parser import parse

def intersection(list1, list2):
    res = []
    idx1 = 0
    while idx1 < len(list1):
        if list1[idx1] in list2:
            res.append(list1[idx1])
        idx1 += 1
    return res

class Revision:
    id = ""
    date = 1
    author = 1
    def __init__(self, date, author, id):
        self.date = date
        self.author = author
        self.id = id
fields = ['snapshot_id', 'id', 'date', 'author']
df = pd.read_csv('/mnt/17volume/data/snapshot_revision_git_1k.csv', usecols=fields)

snapshots = dict(tuple(df.groupby('snapshot_id')))

one_day = 86400
one_week = 604800
one_month = 2628000
with open('/tmp/project-git-metrics.csv', mode='w') as metric_file:
    metric_writer_2 = csv.writer(metric_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    metric_writer_2.writerow(['snapshot_id', 'total_revisions', 'total_authors', 'daily_commits', 'daily_committed_contributors', 'weekly_commits', 'weekly_committed_contributors','monthly_commits', 'monthly_committed_contributors'])
    for id, snapshot in snapshots.items():
        snapshot = snapshot.sort_values(by=['date'])
        min_date = snapshot.date.min()
        max_date = snapshot.date.max()
        if max_date-min_date < 7890000:
            continue
        daily_wnd = []
        weekly_wnd = []
        monthly_wnd = []
        base_date = min_date
        while base_date <= max_date:
            base_date += one_day
            daily_wnd.append(base_date)
        base_date = min_date
        while base_date <= max_date:
            base_date += one_week
            weekly_wnd.append(base_date)
        base_date = min_date
        while base_date <= max_date:
            base_date += one_month
            monthly_wnd.append(base_date)

        total_commits = 0
        authors = []
        
        daily_idx = 0
        daily_commits = []
        daily_contributors = []
        daily_temp_contrib = []
        
        weekly_idx = 0
        weekly_commits = []
        weekly_contributors = []
        weekly_temp_contrib = []
        
        monthly_idx = 0
        monthly_commits = []
        monthly_contributors = []
        monthly_temp_contrib = []
        
        for rev in snapshot.itertuples():
            total_commits += 1
            if rev.author not in authors:
                authors.append(rev.author)
                
            if rev.author not in daily_contributors and rev.date <= daily_wnd[0]:
                daily_contributors.append(rev.author)
            if rev.date <= daily_wnd[daily_idx]:
                if rev.author not in daily_temp_contrib:
                    daily_temp_contrib.append(rev.author)
            else:
                daily_contributors = intersection(daily_contributors, daily_temp_contrib)
                daily_temp_contrib.clear()
                daily_idx += 1
                while daily_idx < len(daily_wnd) and rev.date > daily_wnd[daily_idx]:
                    daily_commits.append(0)
#                     print("day")
                    daily_contributors.clear()
                    daily_idx += 1
                daily_temp_contrib.append(rev.author)
                daily_commits.append(1)
                
            if rev.author not in weekly_contributors and rev.date <= weekly_wnd[0]:
                weekly_contributors.append(rev.author)
            if rev.date <= weekly_wnd[weekly_idx]:
                if rev.author not in daily_temp_contrib:
                    weekly_temp_contrib.append(rev.author)
            else:
                weekly_contributors = intersection(weekly_contributors, weekly_temp_contrib)
                weekly_temp_contrib.clear()
                weekly_idx += 1
                while weekly_idx < len(weekly_wnd) and rev.date > weekly_wnd[weekly_idx]:
                    weekly_commits.append(0)
#                     print("week")
                    weekly_contributors.clear()
                    weekly_idx += 1
                weekly_temp_contrib.append(rev.author)
                weekly_commits.append(1)
            
            if rev.author not in monthly_contributors and rev.date <= monthly_wnd[0]:
                monthly_contributors.append(rev.author)
            if rev.date <= monthly_wnd[monthly_idx]:
                if rev.author not in daily_temp_contrib:
                    monthly_temp_contrib.append(rev.author)
            else:
                monthly_contributors = intersection(monthly_contributors, monthly_temp_contrib)
                monthly_temp_contrib.clear()
                monthly_idx += 1
                while monthly_idx < len(monthly_wnd) and rev.date > monthly_wnd[monthly_idx]:
                    monthly_commits.append(0)
#                     print("month")
                    monthly_contributors.clear()
                    monthly_idx += 1
                monthly_temp_contrib.append(rev.author)
                monthly_commits.append(1)
              

        if len(daily_commits) == 0:
            daily_commits.append(1)
        if len(weekly_commits) == 0:
            weekly_commits.append(1)
        if len(monthly_commits) == 0:
            monthly_commits.append(1)
        if len(monthly_contributors) > 0:
            metric_writer_2.writerow([id, total_commits, len(authors), sum(daily_commits)/len(daily_commits), len(daily_contributors), sum(weekly_commits)/len(weekly_commits), len(weekly_contributors), sum(monthly_commits)/len(monthly_commits), len(monthly_contributors)])